In [ ]:
# Fine-Tuning Afro-XLM-RoBERTa with 5-Fold CV on 80% Train Set + Final Test on 20% Holdout
# Dataset: 477 Offensive, 500 Non-Offensive Samples

"""
Fine-tuning Afro-XLM-R with 5-fold cross-validation on an 80% training split,
followed by final evaluation on a 20% holdout test set.

Dataset (after preprocessing and splitting):
- 477 Offensive samples
- 500 Non-offensive samples

This script assumes you already have:
- train.csv  (80% of data)
- test.csv   (20% holdout)

both stored in a local ./data/ directory.
"""

!pip install transformers datasets torch scikit-learn

import os
import torch
import numpy as np
import pandas as pd
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import precision_recall_fscore_support, accuracy_score, roc_auc_score, matthews_corrcoef
from transformers import Trainer, TrainingArguments, AutoModelForSequenceClassification, AutoTokenizer
from datasets import Dataset
from torch.nn import CrossEntropyLoss

# ---------------------------------------------------------------------
# Configuration
# ---------------------------------------------------------------------
# Base model used for fine-tuning. Replace this with your fine-tuning
# model on Hugging Face, e.g. "Davlan/afro-xlmr-base".

model_name = 'Davlan/afro-xlmr-base'
tokenizer = AutoTokenizer.from_pretrained(model_name)
# Optional: if your experiments use explicit trigger markers, document it:
# SPECIAL_TOKENS = {"additional_special_tokens": ["<TRIGGER>", "</TRIGGER>"]}
# tokenizer.add_special_tokens(SPECIAL_TOKENS)
# tokenizer.add_special_tokens({'additional_special_tokens': ['<TRIGGER>', '</TRIGGER>']})

# Directory where preprocessed CSV files are stored.
# For public code, we assume a local ./data/ folder in the repo.
DATA_DIR = Path("data")

train_df = pd.read_csv('DATA_DIR/training_data2.csv')
test_df = pd.read_csv('DATA_DIR/testing_data2.csv')
train_df['TEXT'] = train_df['TEXT'].astype(str)
test_df['TEXT'] = test_df['TEXT'].astype(str)

def tokenize_batch(batch):
    return tokenizer(batch['TEXT'], padding='max_length', truncation=True, max_length=128)

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    preds = np.argmax(logits, axis=-1)
    precision, recall, f1, _ = precision_recall_fscore_support(labels, preds, average=None, labels=[0, 1])
    return {
        'accuracy': accuracy_score(labels, preds),
        'mcc': matthews_corrcoef(labels, preds),
        'roc_auc': roc_auc_score(labels, torch.softmax(torch.tensor(logits), dim=-1)[:, 1].numpy()),
        'f1_macro': f1.mean(),
        'recall_1': recall[1]
    }

def compute_weighted_loss(model, inputs, return_outputs=False):
    labels = inputs.pop('labels')
    outputs = model(**inputs)
    loss_fct = CrossEntropyLoss(weight=torch.tensor([1.0, 2.0]).to(model.device))
    loss = loss_fct(outputs.logits.view(-1, 2), labels.view(-1))
    return (loss, outputs) if return_outputs else loss

training_args = TrainingArguments(
    output_dir='DATA_DIR/AfroXLMR-80-20-noTrigger',
    per_device_train_batch_size=16,
    per_device_eval_batch_size=64,
    num_train_epochs=5,
    learning_rate=1e-5,
    weight_decay=0.01,
    logging_dir='./logs',
    logging_steps=10,
    save_strategy='no',
    eval_strategy='epoch',
    metric_for_best_model='recall_1',
    greater_is_better=True,
    load_best_model_at_end=False
)

kfold = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
fold_results = []
best_models = []

for fold, (train_idx, val_idx) in enumerate(kfold.split(train_df['TEXT'], train_df['TARGET'])):
    print(f'\n=== Fold {fold + 1}/5 ===')
    train_fold = train_df.iloc[train_idx]
    val_fold = train_df.iloc[val_idx]

    train_dataset = Dataset.from_pandas(train_fold).map(tokenize_batch, batched=True)
    val_dataset = Dataset.from_pandas(val_fold).map(tokenize_batch, batched=True)
    train_dataset = train_dataset.rename_column('TARGET', 'label')
    val_dataset = val_dataset.rename_column('TARGET', 'label')

    label_mapping = {'Non-offensive': 0, 'Offensive': 1}
    train_dataset = train_dataset.map(lambda examples: {'label': [label_mapping[x] for x in examples['label']]}, batched=True)
    val_dataset = val_dataset.map(lambda examples: {'label': [label_mapping[x] for x in examples['label']]}, batched=True)

    train_dataset.set_format(type='torch', columns=['input_ids', 'attention_mask', 'label'])
    val_dataset.set_format(type='torch', columns=['input_ids', 'attention_mask', 'label'])

    model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=2, ignore_mismatched_sizes=True)
    # model.resize_token_embeddings(len(tokenizer))

    trainer = Trainer(
        model=model,
        args=training_args,
        train_dataset=train_dataset,
        eval_dataset=val_dataset,
        compute_metrics=compute_metrics,
    )
    trainer.loss_function = compute_weighted_loss
    trainer.train()
    metrics = trainer.evaluate()
    fold_results.append(metrics)
    best_models.append(model)
    print(metrics)

best_fold_index = np.argmax([r['eval_recall_1'] for r in fold_results])
best_model = best_models[best_fold_index]
print(f'\nBest Fold: Fold {best_fold_index + 1}')

test_dataset = Dataset.from_pandas(test_df).map(tokenize_batch, batched=True)
test_dataset = test_dataset.rename_column('TARGET', 'label')

# Apply label_mapping to test_dataset before setting format
test_dataset = test_dataset.map(lambda examples: {'label': [label_mapping[x] for x in examples['label']]}, batched=True) # Apply label mapping to the test dataset
test_dataset.set_format(type='torch', columns=['input_ids', 'attention_mask', 'label'])

test_trainer = Trainer(
    model=best_model,
    args=TrainingArguments(output_dir='./tmp-test-jw300', per_device_eval_batch_size=64),
    compute_metrics=compute_metrics
)
test_metrics = test_trainer.evaluate(test_dataset)
print('\n=== Test Set Metrics ===')
print(test_metrics)

test_trainer = Trainer(
    model=best_model,
    args=TrainingArguments(output_dir='./tmp-test-afroxlmr', per_device_eval_batch_size=64),
    compute_metrics=compute_metrics
)
test_metrics = test_trainer.evaluate(test_dataset)
print('\n=== Test Set Metrics ===')
print(test_metrics)

save_dir = 'DATA_DIR/AfroXLMR-best-fold-model-trigger-free'
best_model.save_pretrained(save_dir)
tokenizer.save_pretrained(save_dir)
print(f"Best performing model saved to: {save_dir}")


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.2/491.2 kB 19.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 32.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 30.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 14.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 13.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 107.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 10.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/398 [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.08M [00:00<?, ?B/s]

...

{'eval_loss': 0.4296453893184662, 'eval_accuracy': 0.8205128205128205, 'eval_mcc': 0.6410256410256411, 'eval_roc_auc': 0.8791913214990137, 'eval_f1_macro': 0.8205128205128205, 'eval_recall_1': 0.8205128205128205, 'eval_runtime': 0.93, 'eval_samples_per_second': 167.75, 'eval_steps_per_second': 3.226, 'epoch': 5.0}

Best Fold: Fold 2


Map:   0%|          | 0/196 [00:00<?, ? examples/s]

Map:   0%|          | 0/196 [00:00<?, ? examples/s]


=== Test Set Metrics ===
{'eval_loss': 0.38945093750953674, 'eval_model_preparation_time': 0.0027, 'eval_accuracy': 0.8622448979591837, 'eval_mcc': 0.7228973640941718, 'eval_roc_auc': 0.9014675052410901, 'eval_f1_macro': 0.8603214823026368, 'eval_recall_1': 0.8111111111111111, 'eval_runtime': 1.1695, 'eval_samples_per_second': 167.586, 'eval_steps_per_second': 3.42}



=== Test Set Metrics ===
{'eval_loss': 0.38945093750953674, 'eval_model_preparation_time': 0.0029, 'eval_accuracy': 0.8622448979591837, 'eval_mcc': 0.7228973640941718, 'eval_roc_auc': 0.9014675052410901, 'eval_f1_macro': 0.8603214823026368, 'eval_recall_1': 0.8111111111111111, 'eval_runtime': 1.1634, 'eval_samples_per_second': 168.468, 'eval_steps_per_second': 3.438}
Best performing model saved to: /content/drive/My Drive/Colab Notebooks/AfroXLMR-best-fold-model-80-20-noTriggers
